In [3]:
def read_predictions(prediction_dir, concat_mode='concat', per_label=False):
    labels = pd.read_csv(os.path.join(prediction_dir, 'labels.csv'))

    filepaths_train, filepaths_test = [], []
    for filepath in sorted(glob.glob('{}/*'.format(prediction_dir))):
        if filepath.endswith('predictions_test_oof.csv'):
            filepaths_test.append(filepath)

    test_dfs = []
    for filepath in filepaths_test:
        test_dfs.append(pd.read_csv(filepath))
    test_dfs = reduce(lambda df1, df2: pd.merge(df1, df2, on=['id', 'fold_id']), test_dfs)
    test_dfs.columns = _clean_columns(test_dfs, keep_colnames = ['id','fold_id'])

    return train_dfs, test_dfs

def get_fold_xy(test,i):
    #train_split = train[train['fold_id'] != i]
    #valid_split = train[train['fold_id'] == i]
    test_split = test[test['fold_id'] == i]

    #y_train = train_split[label_columns].values
    #y_valid = valid_split[label_columns].values
    #columns_to_drop_train = label_columns + ['id','fold_id']
    #X_train = train_split.drop(columns_to_drop_train, axis=1).values
    #X_valid = valid_split.drop(columns_to_drop_train, axis=1).values

    columns_to_drop_test = ['id','fold_id']
    X_test = test_split.drop(columns_to_drop_test, axis=1).values
    return X_test

In [40]:
import pandas as pd
import numpy as np

pr_file = pd.read_csv('newstageone/13/word2vec_scnn_predictions_test_oof.csv')
test_predicts_list = []
for fold in range(0,10):
    get_fold_xy(pr_file,fold)
    test_predicts_list.append(get_fold_xy(pr_file,fold))
#pr_file.to_csv("newstageone/13/bad_word_logreg_predictions_test_oof.csv", index=False)


IOError: File newstageone/13/word2vec_scnn_predictions_test_oof.csv does not exist

In [39]:
test = pd.read_csv('cleaned_test.csv')
test_predicts = np.ones(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts *= fold_predict

test_predicts **= (1. / len(test_predicts_list))
test_ids = test["id"].values
test_ids = test_ids.reshape((len(test_ids), 1))
CLASSES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
test_predicts = pd.DataFrame(data=test_predicts, columns=CLASSES)
test_predicts["id"] = test_ids
test_predicts = test_predicts[["id"] + CLASSES]
test_predicts.to_csv('newstageone/13/word2vec_scnn_predictions_test_oof.csv', index=False)

In [72]:
test = pd.read_csv('newstageone/OOF/oof20.csv')
test.head()
len(test)

159571

In [1]:
import pandas as pd
import numpy as np
import re
import lightgbm as lgb
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning, module='sklearn')
from sklearn.model_selection import cross_val_score, StratifiedKFold

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score


#######################
# FEATURE ENGINEERING #
#######################
"""
Main function
Input: pandas Series and a feature engineering function
Output: pandas Series
"""
def engineer_feature(series, func, normalize=True):
    feature = series.apply(func)
       
    if normalize:
        feature = pd.Series(z_normalize(feature.values.reshape(-1,1)).reshape(-1,))
    feature.name = func.__name__ 
    return feature

"""
Engineer features
Input: pandas Series and a list of feature engineering functions
Output: pandas DataFrame
"""
def engineer_features(series, funclist, normalize=True):
    features = pd.DataFrame()
    for func in funclist:
        feature = engineer_feature(series, func, normalize)
        features[feature.name] = feature
    return features

"""
Normalizer
Input: NumPy array
Output: NumPy array
"""
scaler = StandardScaler()
def z_normalize(data):
    scaler.fit(data)
    return scaler.transform(data)
    
"""
Feature functions
"""
def asterix_freq(x):
    return x.count('!')/len(x)

def uppercase_freq(x):
    return len(re.findall(r'[A-Z]',x))/len(x)
    
"""
Import submission and OOF files
"""
def get_subs(nums):
    subs = np.hstack([np.array(pd.read_csv("SUB/sub" + str(num) + ".csv")[LABELS]) for num in subnums])
    oofs = np.hstack([np.array(pd.read_csv("OOF/oof" + str(num) + ".csv")[LABELS]) for num in subnums])
    return subs, oofs



In [ ]:
def train_folds(X, y, fold_count, model_list):
      fold_size = len(X) // fold_count
      models = []
      total_meta = []
      auc_list = []
      for fold_id in range(0, fold_count):
          print("FOLD {}".format(fold_id))
          fold_start = fold_size * fold_id
          fold_end = fold_start + fold_size
            
          if fold_id == fold_count - 1:
              fold_end = len(X)

          train_x = np.concatenate([X[:fold_start], X[fold_end:]])
          train_y = np.concatenate([y[:fold_start], y[fold_end:]])

          val_x = X[fold_start:fold_end]
          val_y = y[fold_start:fold_end]
            
          
          model, best_auc = _train_model(model_list[fold_id], train_x, train_y, val_x, val_y,callbacks)
            
          meta = model.predict(val_x, batch_size=128)
          if (fold_id == 0):
              total_meta = meta
          else:
              total_meta = np.concatenate((total_meta, meta), axis=0)
          model_path = os.path.join('models', "model{0}_weights.npy".format(fold_id))
          np.save(model_path, model.get_weights())
          models.append(model)
          auc_list.append(best_auc)

      return models, total_meta, auc_list

def _train_model(model, train_x, train_y, val_x, val_y):
    for label in LABELS:
        model = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
        model.fit(
                train_x,
                train_x[label])
        
        y_pred = model.predict_proba(val_x)[:,1]

    total_auc = 0
    for j in range(6):
        auc = compute_auc(val_y[:, j], y_pred[:, j])
        total_auc += auc

    total_loss /= 6.
    total_auc /= 6.
    return model, total_auc
    
def build_model():
    stackers = []
    for fold in range(0,10):
        stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
        stackers.append(stacker)
    return 

In [ ]:
with timer("Scoring Light GBM"):
    scores = []
    folds = KFold(n_splits=4, shuffle=True, random_state=1)
    lgb_round_dict = defaultdict(int)
    trn_lgbset = lgb.Dataset(csr_trn, free_raw_data=False)
    del csr_trn
    gc.collect()
        
    for class_name in class_names:
        print("Class %s scores : " % class_name)
        class_pred = np.zeros(len(train))
        train_target = train[class_name]
        trn_lgbset.set_label(train_target.values)
            
        lgb_rounds = 500

        for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, train_target)):
            watchlist = [
                trn_lgbset.subset(trn_idx),
                trn_lgbset.subset(val_idx)
            ]
            # Train lgb l1
            model = lgb.train(
                params=params,
                train_set=watchlist[0],
                num_boost_round=lgb_rounds,
                valid_sets=watchlist,
                early_stopping_rounds=50,
                verbose_eval=0
            )
            class_pred[val_idx] = model.predict(trn_lgbset.data[val_idx], num_iteration=model.best_iteration)
            score = roc_auc_score(train_target.values[val_idx], class_pred[val_idx])
                
            # Compute mean rounds over folds for each class
            # So that it can be re-used for test predictions
            lgb_round_dict[class_name] += model.best_iteration
            print("\t Fold %d : %.6f in %3d rounds" % (n_fold + 1, score, model.best_iteration))
            
        print("full score : %.6f" % roc_auc_score(train_target, class_pred))
        scores.append(roc_auc_score(train_target, class_pred))
        train[class_name + "_oof"] = class_pred
        submission[class_name] = lr_pred / folds

    # Save OOF predictions - may be interesting for stacking...
    train[["id"] + class_names + [f + "_oof" for f in class_names]].to_csv("lvl0_lgbm_clean_oof.csv",
                                                                            index=False,
                                                                            float_format="%.8f")

    print('Total CV score is {}'.format(np.mean(scores)))

with timer("Predicting probabilities"):
    # Go through all classes and reuse computed number of rounds for each class
    for class_name in class_names:
        with timer("Predicting probabilities for %s" % class_name):
            train_target = train[class_name]
            trn_lgbset.set_label(train_target.values)
            # Train lgb
            model = lgb.train(
                params=params,
                train_set=trn_lgbset,
                num_boost_round=int(lgb_round_dict[class_name] / folds.n_splits)
            )
            submission[class_name] = model.predict(csr_sub, num_iteration=model.best_iteration)

submission.to_csv("lvl0_lgbm_clean_sub.csv", index=False, float_format="%.8f")

In [39]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score


if __name__ == "__main__":
    
    train = pd.read_csv('train.csv').fillna(' ')
    test = pd.read_csv('test.csv').fillna(' ')
    sub = pd.read_csv('sample_submission.csv')
    submission = pd.DataFrame.from_dict({'id': test['id']})
    submission_oof = train[['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

    INPUT_COLUMN = "comment_text"
    LABELS = train.columns[2:]
    
    # Import submissions and OOF files
    # 29: LightGBM trained on Fasttext (CV: 0.9765, LB: 0.9620)
    # 51: Logistic regression with word and char n-grams (CV: 0.9858, LB: ?)
    # 52: LSTM trained on Fasttext (CV: ?, LB: 0.9851)
    subnums = [1,2,3,4,5,6,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
    subs, oofs = get_subs(subnums)
    
    # Engineer features
    feature_functions = [len, asterix_freq, uppercase_freq]
    features = [f.__name__ for f in feature_functions]
    F_train = engineer_features(train[INPUT_COLUMN], feature_functions)
    F_test = engineer_features(test[INPUT_COLUMN], feature_functions)
    
    train_features = np.hstack([F_train[features].as_matrix(), oofs])
    X_test = np.hstack([F_test[features].as_matrix(), subs]) 
    skf = StratifiedKFold(n_splits=10, shuffle=False)
    
    scores_classes = np.zeros((len(LABELS), 10))
    for j, (class_name) in enumerate(LABELS):
        avreal = train[class_name]
        lr_avpred = np.zeros(train.shape[0])
        #stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
        for i, (train_index, val_index) in enumerate(skf.split(train_features, train[class_name].values)):
            #print(train_index)
            stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
            X_train, X_val = train_features[train_index], train_features[val_index]
            y_train, y_val = train.loc[train_index], train.loc[val_index]
            stacker.fit(X_train, y_train[class_name])
            
            scores_val = stacker.predict_proba(X_val)[:, 1]
            scores_classes[j][i] = roc_auc_score(y_val[class_name], scores_val)
            lr_avpred[val_index] = scores_val
            
            lr_y_pred = stacker.predict_proba(X_test)[:, 1]
            if i > 0:
                lr_fpred = lr_pred + lr_y_pred
            else:
                lr_fpred = lr_y_pred

            lr_pred = lr_fpred
            
        lr_oof_auc = roc_auc_score(avreal, lr_avpred)
        print('\n Average class %s AUC:\t%.6f' % (class_name, np.mean(scores_classes[j])))
        print(' Out-of-fold class %s AUC:\t%.6f' % (class_name, lr_oof_auc))
        submission[class_name] = lr_pred / 10
        submission_oof[class_name] = lr_avpred
    
    print('\n Overall AUC:\t%.6f' % (np.mean(scores_classes)))
    submission.to_csv('lgb_stacktwo_pred.csv', index=False)
    submission_oof.to_csv('lgb_stacktwo_meta.csv', index=False)

    '''
    stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
    
    # Fit and submit
    
    scores = []
    for label in LABELS:
        print(label)
        score = cross_val_score(stacker, X_train, train[label], cv=10, scoring='roc_auc')
        print("AUC:", score)
        scores.append(np.mean(score))
        stacker.fit(X_train, train[label])
        sub[label] = stacker.predict_proba(X_test)[:,1]
    print("CV score:", np.mean(scores))
    
    sub.to_csv("29modelstack.csv", index=False)
    '''


 Average class toxic AUC:	0.987802
 Out-of-fold class toxic AUC:	0.987650


/home/stgc/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



 Average class severe_toxic AUC:	0.991559
 Out-of-fold class severe_toxic AUC:	0.991465

 Average class obscene AUC:	0.995286
 Out-of-fold class obscene AUC:	0.995268

 Average class threat AUC:	0.991848
 Out-of-fold class threat AUC:	0.991016

 Average class insult AUC:	0.989942
 Out-of-fold class insult AUC:	0.989909

 Average class identity_hate AUC:	0.990874
 Out-of-fold class identity_hate AUC:	0.990406

 Overall AUC:	0.991219


In [40]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score
from sklearn.linear_model import LogisticRegression


if __name__ == "__main__":
    
    train = pd.read_csv('train.csv').fillna(' ')
    test = pd.read_csv('test.csv').fillna(' ')
    sub = pd.read_csv('sample_submission.csv')
    submission = pd.DataFrame.from_dict({'id': test['id']})
    submission_oof = train[['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

    INPUT_COLUMN = "comment_text"
    LABELS = train.columns[2:]
    
    # Import submissions and OOF files
    # 29: LightGBM trained on Fasttext (CV: 0.9765, LB: 0.9620)
    # 51: Logistic regression with word and char n-grams (CV: 0.9858, LB: ?)
    # 52: LSTM trained on Fasttext (CV: ?, LB: 0.9851)
    subnums = [1,2,3,4,5,6,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
    subs, oofs = get_subs(subnums)
    
    # Engineer features
    feature_functions = [len, asterix_freq, uppercase_freq]
    features = [f.__name__ for f in feature_functions]
    F_train = engineer_features(train[INPUT_COLUMN], feature_functions)
    F_test = engineer_features(test[INPUT_COLUMN], feature_functions)
    
    train_features = np.hstack([F_train[features].as_matrix(), oofs])
    X_test = np.hstack([F_test[features].as_matrix(), subs]) 
    skf = StratifiedKFold(n_splits=10, shuffle=False)
    
    scores_classes = np.zeros((len(LABELS), 10))
    all_parameters = {
                  'C'             : [1.048113, 0.1930, 0.596362, 0.25595, 0.449843, 0.25595],
                  'tol'           : [0.1, 0.1, 0.046416, 0.0215443, 0.1, 0.01],
                  'solver'        : ['lbfgs', 'newton-cg', 'lbfgs', 'newton-cg', 'newton-cg', 'lbfgs'],
                  'fit_intercept' : [True, True, True, True, True, True],
                  'penalty'       : ['l2', 'l2', 'l2', 'l2', 'l2', 'l2'],
                  'class_weight'  : [None, 'balanced', 'balanced', 'balanced', 'balanced', 'balanced'],
                 }
    for j, (class_name) in enumerate(LABELS):
        avreal = train[class_name]
        lr_avpred = np.zeros(train.shape[0])
        #stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
        for i, (train_index, val_index) in enumerate(skf.split(train_features, train[class_name].values)):
            #print(train_index)
            stacker = LogisticRegression(
                C=all_parameters['C'][j],
                max_iter=200,
                tol=all_parameters['tol'][j],
                solver=all_parameters['solver'][j],
                fit_intercept=all_parameters['fit_intercept'][j],
                penalty=all_parameters['penalty'][j],
                dual=False,
                class_weight=all_parameters['class_weight'][j],
                verbose=0)
            X_train, X_val = train_features[train_index], train_features[val_index]
            y_train, y_val = train.loc[train_index], train.loc[val_index]
            stacker.fit(X_train, y_train[class_name])
            
            scores_val = stacker.predict_proba(X_val)[:, 1]
            scores_classes[j][i] = roc_auc_score(y_val[class_name], scores_val)
            lr_avpred[val_index] = scores_val
            
            lr_y_pred = stacker.predict_proba(X_test)[:, 1]
            if i > 0:
                lr_fpred = lr_pred + lr_y_pred
            else:
                lr_fpred = lr_y_pred

            lr_pred = lr_fpred
            
        lr_oof_auc = roc_auc_score(avreal, lr_avpred)
        print('\n Average class %s AUC:\t%.6f' % (class_name, np.mean(scores_classes[j])))
        print(' Out-of-fold class %s AUC:\t%.6f' % (class_name, lr_oof_auc))
        submission[class_name] = lr_pred / 10
        submission_oof[class_name] = lr_avpred
    
    print('\n Overall AUC:\t%.6f' % (np.mean(scores_classes)))
    submission.to_csv('logisticreg_stacktwo_pred.csv', index=False)
    submission_oof.to_csv('logisticreg_stacktwo_meta.csv', index=False)

    '''
    stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
    
    # Fit and submit
    
    scores = []
    for label in LABELS:
        print(label)
        score = cross_val_score(stacker, X_train, train[label], cv=10, scoring='roc_auc')
        print("AUC:", score)
        scores.append(np.mean(score))
        stacker.fit(X_train, train[label])
        sub[label] = stacker.predict_proba(X_test)[:,1]
    print("CV score:", np.mean(scores))
    
    sub.to_csv("29modelstack.csv", index=False)
    '''


 Average class toxic AUC:	0.987147
 Out-of-fold class toxic AUC:	0.987087


/home/stgc/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



 Average class severe_toxic AUC:	0.991326
 Out-of-fold class severe_toxic AUC:	0.991270

 Average class obscene AUC:	0.994503
 Out-of-fold class obscene AUC:	0.994479

 Average class threat AUC:	0.988594
 Out-of-fold class threat AUC:	0.988375

 Average class insult AUC:	0.988594
 Out-of-fold class insult AUC:	0.988550

 Average class identity_hate AUC:	0.988470
 Out-of-fold class identity_hate AUC:	0.988419

 Overall AUC:	0.989772


In [44]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


if __name__ == "__main__":
    
    train = pd.read_csv('train.csv').fillna(' ')
    test = pd.read_csv('test.csv').fillna(' ')
    sub = pd.read_csv('sample_submission.csv')
    submission = pd.DataFrame.from_dict({'id': test['id']})
    submission_oof = train[['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

    INPUT_COLUMN = "comment_text"
    LABELS = train.columns[2:]
    
    # Import submissions and OOF files
    # 29: LightGBM trained on Fasttext (CV: 0.9765, LB: 0.9620)
    # 51: Logistic regression with word and char n-grams (CV: 0.9858, LB: ?)
    # 52: LSTM trained on Fasttext (CV: ?, LB: 0.9851)
    subnums = [1,2,3,4,5,6,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
    subs, oofs = get_subs(subnums)
    
    # Engineer features
    feature_functions = [len, asterix_freq, uppercase_freq]
    features = [f.__name__ for f in feature_functions]
    F_train = engineer_features(train[INPUT_COLUMN], feature_functions)
    F_test = engineer_features(test[INPUT_COLUMN], feature_functions)
    
    train_features = np.hstack([F_train[features].as_matrix(), oofs])
    X_test = np.hstack([F_test[features].as_matrix(), subs]) 
    skf = StratifiedKFold(n_splits=10, shuffle=False)
    
    scores_classes = np.zeros((len(LABELS), 10))

    for j, (class_name) in enumerate(LABELS):
        avreal = train[class_name]
        lr_avpred = np.zeros(train.shape[0])
        #stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
        for i, (train_index, val_index) in enumerate(skf.split(train_features, train[class_name].values)):
            #print(train_index)
            n_estimators = 200
            stacker = clf = XGBClassifier(n_estimators=n_estimators,
                        max_depth=4,
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=.8, 
                        colsample_bytree=.8,
                        gamma=1,
                        reg_alpha=0,
                        reg_lambda=1,
                        nthread=2)
            X_train, X_val = train_features[train_index], train_features[val_index]
            y_train, y_val = train.loc[train_index], train.loc[val_index]
            stacker.fit(X_train, y_train[class_name])
            
            scores_val = stacker.predict_proba(X_val)[:, 1]
            scores_classes[j][i] = roc_auc_score(y_val[class_name], scores_val)
            lr_avpred[val_index] = scores_val
            
            lr_y_pred = stacker.predict_proba(X_test)[:, 1]
            if i > 0:
                lr_fpred = lr_pred + lr_y_pred
            else:
                lr_fpred = lr_y_pred

            lr_pred = lr_fpred
            
        lr_oof_auc = roc_auc_score(avreal, lr_avpred)
        print('\n Average class %s AUC:\t%.6f' % (class_name, np.mean(scores_classes[j])))
        print(' Out-of-fold class %s AUC:\t%.6f' % (class_name, lr_oof_auc))
        submission[class_name] = lr_pred / 10
        submission_oof[class_name] = lr_avpred
    
    print('\n Overall AUC:\t%.6f' % (np.mean(scores_classes)))
    submission.to_csv('XGB_stacktwo_pred.csv', index=False)
    submission_oof.to_csv('XGB_stacktwo_meta.csv', index=False)

    '''
    stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
    
    # Fit and submit
    
    scores = []
    for label in LABELS:
        print(label)
        score = cross_val_score(stacker, X_train, train[label], cv=10, scoring='roc_auc')
        print("AUC:", score)
        scores.append(np.mean(score))
        stacker.fit(X_train, train[label])
        sub[label] = stacker.predict_proba(X_test)[:,1]
    print("CV score:", np.mean(scores))
    
    sub.to_csv("29modelstack.csv", index=False)
    '''


 Average class toxic AUC:	0.987652
 Out-of-fold class toxic AUC:	0.987478


/home/stgc/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



 Average class severe_toxic AUC:	0.991304
 Out-of-fold class severe_toxic AUC:	0.991147

 Average class obscene AUC:	0.995268
 Out-of-fold class obscene AUC:	0.995247

 Average class threat AUC:	0.989995
 Out-of-fold class threat AUC:	0.989601

 Average class insult AUC:	0.989765
 Out-of-fold class insult AUC:	0.989735

 Average class identity_hate AUC:	0.990650
 Out-of-fold class identity_hate AUC:	0.990048

 Overall AUC:	0.990772


In [52]:
def golden_features(data):
    df = pd.DataFrame()
    df['total_length'] = data['comment_text'].apply(len)
    df['capitals'] = data['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
    df['num_exclamation_marks'] = data['comment_text'].apply(lambda comment: comment.count('!'))
    df['num_question_marks'] = data['comment_text'].apply(lambda comment: comment.count('?'))
    df['num_punctuation'] = data['comment_text'].apply(
        lambda comment: sum(comment.count(w) for w in '.,;:'))
    df['num_symbols'] = data['comment_text'].apply(
        lambda comment: sum(comment.count(w) for w in '*&$%'))
    df['num_words'] = data['comment_text'].apply(lambda comment: len(comment.split()))
    df['num_unique_words'] = data['comment_text'].apply(
        lambda comment: len(set(w for w in comment.split())))
    df['words_vs_unique'] = df['num_unique_words'] / df['num_words']
    df['num_smilies'] = data['comment_text'].apply(
        lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
    return df

In [57]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score
from sklearn.linear_model import LogisticRegression

def get_subs(nums):
    subs = np.hstack([np.array(pd.read_csv("SUB_two/sub" + str(num) + ".csv")[LABELS]) for num in subnums])
    oofs = np.hstack([np.array(pd.read_csv("OOF_two/oof" + str(num) + ".csv")[LABELS]) for num in subnums])
    return subs, oofs

if __name__ == "__main__":
    
    train = pd.read_csv('train.csv').fillna(' ')
    test = pd.read_csv('test.csv').fillna(' ')
    sub = pd.read_csv('sample_submission.csv')
    submission = pd.DataFrame.from_dict({'id': test['id']})
    submission_oof = train[['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

    INPUT_COLUMN = "comment_text"
    LABELS = train.columns[2:]
    
    # Import submissions and OOF files
    # 29: LightGBM trained on Fasttext (CV: 0.9765, LB: 0.9620)
    # 51: Logistic regression with word and char n-grams (CV: 0.9858, LB: ?)
    # 52: LSTM trained on Fasttext (CV: ?, LB: 0.9851)
    subnums = [1,2,3]
    subs, oofs = get_subs(subnums)
    
    # Engineer features
    feature_functions = [len, asterix_freq, uppercase_freq]
    features = [f.__name__ for f in feature_functions]
    F_train = engineer_features(train[INPUT_COLUMN], feature_functions)
    F_test = engineer_features(test[INPUT_COLUMN], feature_functions)
    
    

    train_features_pri = np.hstack([F_train[features].as_matrix(), oofs])
    X_test_pri = np.hstack([F_test[features].as_matrix(), subs]) 
    
    gold_F = ('total_length', 'capitals', 'caps_vs_length', 'num_exclamation_marks',
            'num_question_marks', 'num_punctuation', 'num_words', 'num_unique_words',
            'words_vs_unique', 'num_smilies', 'num_symbols')
    gold_Feature = [g for g in gold_F]
    
    G_train = golden_features(train)
    G_test = golden_features(test)
    
    train_features = np.hstack([G_train[gold_Feature].as_matrix(), train_features_pri])
    X_test = np.hstack([G_test[gold_Feature].as_matrix(), X_test_pri])
    
    
    skf = StratifiedKFold(n_splits=10, shuffle=False)
    
    scores_classes = np.zeros((len(LABELS), 10))

    for j, (class_name) in enumerate(LABELS):
        avreal = train[class_name]
        lr_avpred = np.zeros(train.shape[0])
        #stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
        for i, (train_index, val_index) in enumerate(skf.split(train_features, train[class_name].values)):
            #print(train_index)
            stacker = stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
            X_train, X_val = train_features[train_index], train_features[val_index]
            y_train, y_val = train.loc[train_index], train.loc[val_index]
            stacker.fit(X_train, y_train[class_name])
            
            scores_val = stacker.predict_proba(X_val)[:, 1]
            scores_classes[j][i] = roc_auc_score(y_val[class_name], scores_val)
            lr_avpred[val_index] = scores_val
            
            lr_y_pred = stacker.predict_proba(X_test)[:, 1]
            if i > 0:
                lr_fpred = lr_pred + lr_y_pred
            else:
                lr_fpred = lr_y_pred

            lr_pred = lr_fpred
            
        lr_oof_auc = roc_auc_score(avreal, lr_avpred)
        print('\n Average class %s AUC:\t%.6f' % (class_name, np.mean(scores_classes[j])))
        print(' Out-of-fold class %s AUC:\t%.6f' % (class_name, lr_oof_auc))
        submission[class_name] = lr_pred / 10
        submission_oof[class_name] = lr_avpred
    
    print('\n Overall AUC:\t%.6f' % (np.mean(scores_classes)))
    submission.to_csv('lgbm_stack_final_pred.csv', index=False)
    submission_oof.to_csv('LGBM_stack_final_meta.csv', index=False)

    
    stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
    
    # Fit and submit
    
    scores = []
    for label in LABELS:
        print(label)
        score = cross_val_score(stacker, train_features, train[label], cv=10, scoring='roc_auc')
        print("AUC:", score)
        scores.append(np.mean(score))
        stacker.fit(train_features, train[label])
        sub[label] = stacker.predict_proba(X_test)[:,1]
    print("CV score:", np.mean(scores))
    
    sub.to_csv("TRAIN_ALL_STACK_LOG_REG.csv", index=False)
    


 Average class toxic AUC:	0.988588
 Out-of-fold class toxic AUC:	0.988451


/home/stgc/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



 Average class severe_toxic AUC:	0.991879
 Out-of-fold class severe_toxic AUC:	0.991808

 Average class obscene AUC:	0.995325
 Out-of-fold class obscene AUC:	0.995282

 Average class threat AUC:	0.990770
 Out-of-fold class threat AUC:	0.990069

 Average class insult AUC:	0.989822
 Out-of-fold class insult AUC:	0.989714

 Average class identity_hate AUC:	0.990839
 Out-of-fold class identity_hate AUC:	0.990254

 Overall AUC:	0.991204
toxic
('AUC:', array([0.98889795, 0.98861989, 0.98971198, 0.9868848 , 0.98862791,
       0.9890668 , 0.98846785, 0.98917377, 0.98753618, 0.9888971 ]))
severe_toxic
('AUC:', array([0.99273583, 0.99111359, 0.99293898, 0.99054152, 0.99058346,
       0.99194787, 0.99232698, 0.99152614, 0.99322736, 0.99184623]))
obscene
('AUC:', array([0.99580642, 0.99464038, 0.99561256, 0.99475678, 0.9959103 ,
       0.99596112, 0.99524442, 0.99531185, 0.99552059, 0.99448298]))
threat
('AUC:', array([0.98517704, 0.99588571, 0.99611617, 0.98161418, 0.98804791,
       0.98691385,

In [41]:
test = pd.read_csv('SUB_two/logisticreg_stacktwo_pred.csv')
test.head()
#columns_to_drop_test = ['toxic','severe_toxic','threat','insult','identity_hate','obscene']
#columns_to_drop_test = ['comment_text']

#test = test.drop(columns_to_drop_test, axis=1)

#test = test.rename(columns={'toxic_oof': 'toxic', 'severe_toxic_oof': 'severe_toxic', 'obscene_oof': 'obscene', 'threat_oof': 'threat', 'insult_oof': 'insult', 'identity_hate_oof': 'identity_hate'})
#test.to_csv("29modelstack.csv", index=False)


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999646,0.992186,0.999660,0.849823,0.996217,0.997916
1,0000247867823ef7,0.006185,0.001481,0.008564,0.016933,0.017819,0.020447
2,00013b17ad220c46,0.006356,0.001386,0.009035,0.017988,0.016962,0.018510
3,00017563c3f7919a,0.004452,0.001463,0.008542,0.027373,0.013220,0.017579
4,00017695ad8997eb,0.005858,0.001344,0.010354,0.025551,0.014992,0.019761


In [58]:
subnums = [4]
subs, oofs = get_subs(subnums)


IOError: File SUB_two/sub4.csv does not exist

In [ ]:
test = pd.read_csv('OOF/oof28.csv')
